In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/data-science-job-posting-on-glassdoor/Uncleaned_DS_jobs.csv')


In [ ]:
df.drop('index', inplace=True, axis=1)
df.head()

## Things i am going to do :
drop unwanted column<br>
make salary column int<br>
extract info out of job descripiton<br>
remove int from company name<br>
get a state column from value of location<br>
create some features



In [ ]:
df['Salary Estimate'].unique()

In [ ]:
df['Salary Estimate'].apply(lambda x: x.split('(')[0]).unique()

In [ ]:
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.split('(')[0])

In [ ]:
df['Salary Estimate'].apply(lambda x: x.replace('K','').replace('$','')).unique()

In [ ]:
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: x.replace('K','').replace('$',''))

In [ ]:
df['min_salary'] = df['Salary Estimate'].apply(lambda x: int(x.split('-')[0]))
df['max_salary'] = df['Salary Estimate'].apply(lambda x: int(x.split('-')[1]))
df.head()

In [ ]:
df['avg_salary'] = (df.min_salary+df.max_salary)//2
df.head()

In [ ]:
df['Company Name'] = df['Company Name'].apply(lambda x: x.split('\n')[0])
df.head()

In [ ]:
df.sort_values(by='Rating', ascending=True)

In [ ]:
df[df['Rating']==-1.0].shape

In [ ]:
df.Rating.dtype

In [ ]:
import numpy as np
df.Rating = np.where(df.Rating==-1.0,0,df.Rating)

In [ ]:
df.sort_values(by='Rating', ascending=True)

In [ ]:
df['job_state'] = df['Location'].apply(lambda x: x.split(',')[-1])
df.head()

In [ ]:
df['same_state'] = df.apply(lambda x: 1 if x.Location==x.Headquarters else 0,axis=1)
df.head()

In [ ]:
df.Founded

In [ ]:
df[df['Founded']==-1].shape

In [ ]:
df['company_age'] = df.Founded.apply(lambda x: x if x<1 else 2021-x)
df.head()

In [ ]:
df[df['company_age']==-1].shape

In [ ]:
df.company_age = np.where(df.company_age==-1.0,0,df.Rating)
df[df['company_age']==-1].shape

In [ ]:
df['Job Description'][2].split('\n')

In [ ]:
df['python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
df['excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
df['hadoop'] = df['Job Description'].apply(lambda x: 1 if 'hadoop' in x.lower() else 0)
df['spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
df['aws'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
df['tableau'] = df['Job Description'].apply(lambda x: 1 if 'tableau' in x.lower() else 0)
df['big_data'] = df['Job Description'].apply(lambda x: 1 if 'big data' in x.lower() else 0)
df.head()

In [ ]:
df['Job Title'].value_counts()

In [ ]:

def title_simplifier(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'machine learning' in title.lower():
        return 'mle'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else:
        return 'na'
    
def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'sr' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
            return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower():
        return 'jr'
    else:
        return 'na'


In [ ]:
df['job_simp'] = df['Job Title'].apply(title_simplifier)

df.job_simp.value_counts()

In [ ]:
df['seniority'] = df['Job Title'].apply(seniority)
df.seniority.value_counts()

In [ ]:
df.job_state.value_counts()

In [ ]:
df = df[~((df['job_state']=='Remote') | (df['job_state']=='Utah') | (df['job_state']=='New Jersey') | (df['job_state']=='Texas') | (df['job_state']=='California'))]
df.job_state = df.job_state.apply(lambda x: x.replace('United States','US'))
df.job_state.value_counts()

In [ ]:
df.head()

In [ ]:
df.drop(['Founded','Competitors'], axis=1,inplace=True)

In [ ]:
df = df.apply(lambda x: x.strip() if isinstance(x,str) else x)

In [ ]:
df.to_csv('Cleaned_DS_Jobs.csv', index=False)